In [15]:
import pandas as pd
import nltk
import string
import re
import numpy as np
from tqdm import tqdm
import pandas as pd
from nltk.stem import WordNetLemmatizer 
from nltk.tokenize import word_tokenize
import preprocessor as p
from nltk.corpus import stopwords
from tqdm import tqdm

In [192]:
# Function to get All caps count from the tweets
# pass tweets as it is
def FuncAllCapsCount(GivenTweets):
    AllCapsCount=np.zeros(len(GivenTweets))
    for i in tqdm(range(len(GivenTweets))):
        tweet=nltk.word_tokenize(GivenTweets[i])
        for word in tweet:
            if(word!="I" and word!="USER" and re.match("^[A-Z]+$",word)):
                AllCapsCount[i]+=1
    return(AllCapsCount)

In [131]:
# Function to get COntinuous Punctuation (???,??!,!!) Count and check if last token was a ? or !
# pass tweets as it is
def FuncPunctuationsCount(GivenTweets):
    ContPunctuationCount=np.zeros(len(GivenTweets))
    LastTokenPunctuation=np.zeros(len(GivenTweets))
    for i in tqdm(range(len(GivenTweets))):
        tweet=nltk.word_tokenize(GivenTweets[i])
        token=0
        while token<len(tweet):
            if(tweet[token]=="?" or tweet[token]=="!"):
                index=token+1
                while(index< len(tweet) and (tweet[index]=="?" or tweet[index]=="!")):
                    index+=1
                if(index-token>1):
                    ContPunctuationCount[i]+=1
                    token=index
            token+=1
        if(tweet[len(tweet)-1]=="?" or tweet[len(tweet)-1]=="!"):
            LastTokenPunctuation[i]+=1
    
    return(ContPunctuationCount,LastTokenPunctuation)

In [179]:
# Fucntion to get HashTag count from the tweets
# pass tweets as it is
def FuncHashtagsCount(GivenTweets):    
    HashTagsCount=np.zeros(len(GivenTweets))
    for i in tqdm(range(len(GivenTweets))):
        tweet=GivenTweets[i].split(" ")
        for word in tweet:
            if(re.match("^#[a-zA-Z][a-zA-Z0-9]*",word)):
                HashTagsCount[i]+=1
    return(HashTagsCount)

In [198]:
# NRC-Hashtag-Emotion-Lexicon
def Func_NRC_Hashtag_Emotion(GivenTweets):
    d_emotion=pd.read_csv("./lexicons/5. NRC-Hashtag-Emotion-Lexicon-v0.2.txt", sep='\t', names=['emotion','word','score'], header=None)
    dic_dEmo={}
    for i in range(len(d_emotion['word'])):
        if d_emotion['emotion'][i]=="anger":
            dic_dEmo[d_emotion['word'][i]]=d_emotion['score'][i]
    aggEmoHashtags=np.zeros(len(GivenTweets))
    for i in range(len(GivenTweets)):
        tweet=GivenTweets[i]
        hashtags = [j  for j in tweet.split() if j.startswith("#") ]
        for tag in hashtags:
            word=tag.lower()
            if word in dic_dEmo:
                aggEmoHashtags[i]+=dic_dEmo[word]
            elif word[1:] in dic_dEmo:
                aggEmoHashtags[i]+=dic_dEmo[word[1:]]
    return(aggEmoHashtags)
    
    #     Vector aggEmoHashtags conatains Aggregate emotion score (Hashtags)
    #     Adding features into dataframe


In [214]:
# NRC-Hashtag-Emotion-Lexicon
def Func_NRC_10_Expanded_Emotion(GivenTweets):
    d_emotion=pd.read_csv("./lexicons/6. NRC-10-expanded.csv", sep='\t')
    dic_dEmo={}
    for i in range(len(d_emotion['word'])):
        dic_dEmo[d_emotion['word'][i]]=d_emotion['anger'][i]
    aggScoreEmo=np.zeros(len(GivenTweets))
    for i in range(len(GivenTweets)):
#         print(i)
        tweet=tweets_train[i]
        for word in word_tokenize(tweet):
            if word in dic_dEmo:
#                 print(word)
                aggScoreEmo[i]+=dic_dEmo[word]
    return(aggScoreEmo)

In [225]:
def Func_NRC_word_emotion_lexicon(GivenTweets):
    emotion_count=np.zeros(len(tweets_train))
    emotions_df=pd.read_csv("./lexicons/8. NRC-word-emotion-lexicon.txt", sep='\t', names=['word','emotion','score'], header=None)
    # This set contains words with Emotion X=1 in above dataframe
    set_emo=set()
    for i in range(len(emotions_df['word'])):
        if emotions_df['emotion'][i]=="anger" and emotions_df['score'][i]==1:
            set_emo.add(emotions_df['word'][i])

    for i in range(len(tweets_train)):
        tweet=word_tokenize(tweets_train[i])
        for word in tweet:
            word_lower=word.lower()
            if word_lower in set_emo:
                emotion_count[i]+=1
    return(emotion_count)

# Preprocess main function

In [151]:
def preprocess_text(tweet, flag_stemm=False, flag_lemm=False, stop_words_list=None):
    
    ## clean (convert to lowercase and remove punctuations and characters and then strip)
    tweet = re.sub(r'[^\w\s]', '', str(tweet).lower().strip())
    ## Tokenize (convert from string to list) and remove the stop words
    tokenize_tweet = tweet.split()

    if stop_words_list is not None:
        tokenize_tweet = [word for word in tokenize_tweet if word not in stop_words_list]


    ## Stemming (remove -ing, -ly, ...)
    if flag_stemm == True:
        ps = nltk.stem.porter.PorterStemmer()
        tokenize_tweet = [ps.stem(word) for word in tokenize_tweet]

    ## Lemmatisation (convert the word into root word)
    if flag_lemm == True:
        lem = nltk.stem.wordnet.WordNetLemmatizer()
        tokenize_tweet = [lem.lemmatize(word) for word in tokenize_tweet]
#     print(tokenize_tweet)
    ## back to string from list
    tweet = " ".join(tokenize_tweet)
    return tweet

# Preprocess caller function

In [103]:
def preprocess_text_caller(Tweets_List):
    stop_words_list=stopwords.words('english')
    for i in (range(len(Tweets_List))):
        Tweets_List[i]=preprocess_text(Tweets_List[i])

In [222]:
def feature_extraction(Tweets_List):
    #Counting the Capital Words
    df_feature=pd.DataFrame(FuncAllCapsCount(Tweets_List))
    df_feature.rename(columns={0:"all_caps_count"},inplace=True)
#     df_feature["all_caps_count"]=df_feature["all_caps_count"].astype('int32')

    #Counting The Punctuations
    ContPunctuationCount,LastTokenPunctuation=FuncPunctuationsCount(Tweets_List)
    df_feature["Cont_Punc_Count"]=pd.DataFrame(ContPunctuationCount)
    df_feature["Last_Token_Punc"]=pd.DataFrame(LastTokenPunctuation)
    
    #Count the hash tags
    df_feature["HashTag_Count"]=pd.DataFrame(FuncHashtagsCount(Tweets_List))
    df_feature["Anger word count"]=pd.DataFrame(Func_NRC_word_emotion_lexicon(Tweets_List))
    
    df_feature=df_feature.astype('int32')
    #Add integer features before this
    
    
    df_feature["NRC_Hashtag"]=pd.DataFrame(Func_NRC_Hashtag_Emotion(Tweets_List))
    
    df_feature["NRC_10_Expanded"]=pd.DataFrame(Func_NRC_10_Expanded_Emotion(Tweets_List))

    return(df_feature)

# Use this for preprocessing

In [105]:
df_training=pd.DataFrame(pd.read_csv("olid-training-v1.0.tsv",sep="\t"))
df_training['tweet_original']=df_training['tweet']
preprocess_text_caller(df_training['tweet'])

<ipython-input-103-2b67d72cee89>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  Tweets_List[i]=preprocess_text(Tweets_List[i])


In [229]:
a

,all_caps_count,Cont_Punc_Count,Last_Token_Punc,HashTag_Count,Anger word count,NRC_Hashtag,NRC_10_Expanded
0,0,0,0,0,0,0.000000,0.735859
1,2,1,0,2,0,0.000000,0.335047
2,5,0,0,5,0,0.368004,0.796653
3,0,0,0,0,1,0.000000,0.269213
4,0,0,0,0,0,0.000000,0.478728
...,...,...,...,...,...,...,...
95,0,0,0,0,0,0.000000,2.963152
96,0,0,1,0,0,0.000000,0.942392
97,0,0,0,0,0,0.000000,0.402854
98,1,0,0,0,0,0.000000,0.164888


In [226]:
a=feature_extraction(df_training['tweet_original'][0:100])

100%|██████████| 100/100 [00:00<00:00, 22948.54it/s]


In [196]:
# count=0
# for i in df_training['tweet_original'][0:100]:
#     print(count,i)
#     count+=1

In [219]:
tweets_train=df_training['tweet_original'][0:100]